# `Nuqleon.Memory`

Provides object pools, function memoization, and caching utilities.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [1]:
#r "bin/Debug/net50/Nuqleon.Memory.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [1]:
#r "nuget:Nuqleon.Memory,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [1]:
System.Diagnostics.Debugger.Launch();

## Using object pools

Object pools can be used to reduce the overhead of allocating fresh objects. This library supports object pooling for arbitrary types but also have built-in support for commonly used types, such as `StringBuilder` and various collection types.

### Using an object pool for a well-known collection type

First, we'll explore support for built-in types by having a look at pooling for `Stack<T>` objects. Support for other types is completely analogous.

#### Step 1 - Create a pool

The first step is to create a pool using one of the `Create` static method overloads on the pool type. In this example, we'll use `StackPool<T>` and create a pool that can grow up to `8` instances.

In [1]:
var pool = StackPool<int>.Create(size: 8);

#### Step 2 - Inspect the pool using `DebugView`

At any time we can have a look at the pool's internals using the `DebugView` property. This shows statistics of the pool, as well as call stacks for allocations and deallocations in `DEBUG` builds.

**Note:** Because the `DebugView` contains quite lengthy stack traces due to invocations taking place in .NET Interactive underneath the Notebook, we use a simple helper function to trim stack traces below. In regular use cases outside Notebooks, `DebugView` is typically accessed in the Visual Studio debugger Watch window.

In [1]:
using System.IO;
using System.Text.RegularExpressions;

// Regular expression to match stack trace lines with any amount of leading whitespace.
var isStackTraceLine = new Regex("^([ \t]*)at (.*)$");

// Eat our own object pooling dogfood here as well :-).
var stringBuilderPool = StringBuilderPool.Create(size: 8);

string TrimDebugView(string debugView)
{
    using (var sb = stringBuilderPool.New())
    using (var sr = new StringReader(debugView))
    {
        var skip = false;
        string line;

        while ((line = sr.ReadLine()) != null)
        {
            var match = isStackTraceLine.Match(line);

            if (skip && !match.Success)
            {
                skip = false;
            }

            if (!skip)
            {
                if (match.Success && match.Groups[2].Value.StartsWith("Submission"))
                {
                    sb.StringBuilder.AppendLine(match.Groups[1].Value + "at <Notebook>");
                    skip = true;
                }
                else
                {
                    sb.StringBuilder.AppendLine(line);
                }
            }
        }

        return sb.StringBuilder.ToString();
    }
}

void PrintDebugView()
{
    Console.WriteLine(TrimDebugView(pool.DebugView));
}

PrintDebugView();


System.Collections.Generic.StackPool`1+Impl0[System.Int32] : ObjectPoolBase<System.Collections.Generic.PooledStack`1[System.Int32]>

Creation trace
--------------

21/03/04 16:58:31.3985714 [00:00:01.6164524 ago] @ ~13 () in 1 (Microsoft.DotNet.Interactive.App)
   at System.Memory.ObjectPoolBase`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Impl0..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Create(Int32 size)
   at <Notebook>


Statistics
----------

  # slots:   8
  # standby: 0
  # allocs:  0
  # frees:   0
  # create:  0
  # drops:   0


Pool entries
------------

  [0] = <empty>
  [1] = <empty>
  [2] = <empty>
  [3] = <empty>
  [4] = <empty>
  [5] = <empty>
  [6] = <empty>
  [7] = <empty>


Tracked objects
---------------






#### Step 3 - Allocate an object from the pool

One way to allocate an object from the pool is by using `Allocate`. Once we're done using the object, we call `Free`. This is typically done in a safe manner, e.g. using a `try...finally...` statement. In case an object is not returned to the pool, it will just get garbage collected and the pool's performance will be degraded. However, there won't be a memory leak.

In [1]:
PooledStack<int> stack = pool.Allocate();

PrintDebugView();

try
{
    // Use the object here.
    stack.Push(1);
}
finally
{
    pool.Free(stack);
}

System.Collections.Generic.StackPool`1+Impl0[System.Int32] : ObjectPoolBase<System.Collections.Generic.PooledStack`1[System.Int32]>

Creation trace
--------------

21/03/04 16:58:31.3985714 [00:00:08.3823493 ago] @ ~13 () in 1 (Microsoft.DotNet.Interactive.App)
   at System.Memory.ObjectPoolBase`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Impl0..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Create(Int32 size)
   at <Notebook>


Statistics
----------

  # slots:   8
  # standby: 0
  # allocs:  1
  # frees:   0
  # create:  1
  # drops:   0


Pool entries
------------

  [0] = <empty>
  [1] = <empty>
  [2] = <empty>
  [3] = <empty>
  [4] = <empty>
  [5] = <empty>
  [6] = <empty>
  [7] = <empty>


Tracked objects
---------------

  #1 - checked out
  
    Stats
    ~~~~~

      Usage count =  0
      
    History
    ~~~~~~~

      Alloc - 21/03/04 16:58:39.7808835 [00:00:00.0080656 ago] @ 

Now that we've returned the object back to the pool, let's inspect the pool again.

In [1]:
PrintDebugView();

System.Collections.Generic.StackPool`1+Impl0[System.Int32] : ObjectPoolBase<System.Collections.Generic.PooledStack`1[System.Int32]>

Creation trace
--------------

21/03/04 16:58:31.3985714 [00:00:15.1071969 ago] @ ~13 () in 1 (Microsoft.DotNet.Interactive.App)
   at System.Memory.ObjectPoolBase`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Impl0..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Create(Int32 size)
   at <Notebook>


Statistics
----------

  # slots:   8
  # standby: 1
  # allocs:  1
  # frees:   1
  # create:  1
  # drops:   0

  avg use time: 00:00:00.0101310
  max use time: 00:00:00.0101310


Pool entries
------------

  [0] = #1
  [1] = <empty>
  [2] = <empty>
  [3] = <empty>
  [4] = <empty>
  [5] = <empty>
  [6] = <empty>
  [7] = <empty>


Tracked objects
---------------

  #1 - standby
  
    Stats
    ~~~~~

      Usage count =  1
      Max use time = 00:00:00.0101310
 

#### Step 4 - An alternative way to allocate from the pool

An alternative way to allocate an object from the pool is by using `New` which returns a holder object that implements `IDisposable` and can be used with a `using` statement. This makes it easier to ensure returning the object to the pool, even in exceptional circumstances.

In [1]:
using (PooledStackHolder<int> h = pool.New())
{
    PrintDebugView();

    var s = h.Stack;

    // Use the object here.
    s.Push(1);
}

System.Collections.Generic.StackPool`1+Impl0[System.Int32] : ObjectPoolBase<System.Collections.Generic.PooledStack`1[System.Int32]>

Creation trace
--------------

21/03/04 16:58:31.3985714 [00:00:24.0868703 ago] @ ~13 () in 1 (Microsoft.DotNet.Interactive.App)
   at System.Memory.ObjectPoolBase`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Impl0..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Create(Int32 size)
   at <Notebook>


Statistics
----------

  # slots:   8
  # standby: 0
  # allocs:  2
  # frees:   1
  # create:  1
  # drops:   0

  avg use time: 00:00:00.0101310
  max use time: 00:00:00.0101310


Pool entries
------------

  [0] = <empty>
  [1] = <empty>
  [2] = <empty>
  [3] = <empty>
  [4] = <empty>
  [5] = <empty>
  [6] = <empty>
  [7] = <empty>


Tracked objects
---------------

  #1 - checked out
  
    Stats
    ~~~~~

      Usage count =  1
      Max use time = 00:00:00.

Now that we've returned the object back to the pool, let's inspect the pool again.

In [1]:
PrintDebugView();

System.Collections.Generic.StackPool`1+Impl0[System.Int32] : ObjectPoolBase<System.Collections.Generic.PooledStack`1[System.Int32]>

Creation trace
--------------

21/03/04 16:58:31.3985714 [00:00:28.0115706 ago] @ ~13 () in 1 (Microsoft.DotNet.Interactive.App)
   at System.Memory.ObjectPoolBase`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Impl0..ctor(Int32 size)
   at System.Collections.Generic.StackPool`1.Create(Int32 size)
   at <Notebook>


Statistics
----------

  # slots:   8
  # standby: 1
  # allocs:  2
  # frees:   2
  # create:  1
  # drops:   0

  avg use time: 00:00:00.0078948
  max use time: 00:00:00.0101310


Pool entries
------------

  [0] = #1
  [1] = <empty>
  [2] = <empty>
  [3] = <empty>
  [4] = <empty>
  [5] = <empty>
  [6] = <empty>
  [7] = <empty>


Tracked objects
---------------

  #1 - standby
  
    Stats
    ~~~~~

      Usage count =  2
      Max use time = 00:00:00.0101310
 

### Use an object pool for a custom type

To understand how object pools work at the next level of detail, let's use object pooling for a custom object type. There are a few ways to work with object pools, including deriving from `ObjectPoolBase<T>` or by using `ObjectPool<T>` directly. We'll explore the latter.

#### Step 1 - Create a custom type

To illustrate the behavior of pooling, we'll start by defining a custom type for which we'll pool instances.

In [1]:
class MyObject
{
    // Demonstrates the expense of the object which may warrant pooling to reuse these array allocatons.
    private readonly int[] _values = new int[16];

    public int this[int i]
    {
        get => _values[i];
        set => _values[i] = value;
    }

    public override string ToString() => string.Join(", ", _values);
}

#### Step 2 - Create an object pool

Next, rather than allocating `MyObject` instances directly, we'll use an `ObjectPool<MyObject>` which gets parameterized on a `Func<MyObject>` factory. For illustration purposes, we'll include a side-effect to indicate an allocation has taken place. The second `size` parameter passed to the constructor indicates the maximum number of instances held by the pool.

In [1]:
using System.Memory;

var myPool = new ObjectPool<MyObject>(() =>
{
    var res = new MyObject();
    Console.WriteLine($"Allocated a new MyObject instance. Hash code = {res.GetHashCode()}");
    return res;
}, size: 4);

#### Step 3 - Use the pool to allocate instances

Let's now use our freshly created pool to allocate an instance of `MyObject` and witness the invocation of the factory.

In [1]:
MyObject myObj1 = myPool.Allocate();

myObj1[0] = 42;

Console.WriteLine(myObj1);

Allocated a new MyObject instance. Hash code = 12239935


42, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


#### Step 4 - Create a second object from the pool

When we request another instance of `MyObject` from the pool while `myObj1` is in use, another allocation will take place. In this example, we use `New` which returns a `PooledObject<T>` which implements `IDisposable` to return the object to the pool. By using a `using` statement, the object gets returned to the pool automatically.

In [1]:
using (PooledObject<MyObject> myObj2 = myPool.New())
{
    myObj2.Object[0] = 43;

    Console.WriteLine(myObj2.Object);
}

Allocated a new MyObject instance. Hash code = 31965818


43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


#### Step 5 - Witness the reuse of objects

To illustrate the reuse of objects from the pool, let's allocate yet another object from the pool. Because we still are holding on to `myObj1` but have used and released `myObj2`, the latter object can be reused. Note that the hash code of the object returned from the pool matches `myObj2` in the cell above.

In [1]:
using (PooledObject<MyObject> myObj3 = myPool.New())
{
    Console.WriteLine($"Hash code = {myObj3.Object.GetHashCode()}");

    Console.WriteLine(myObj3.Object);
}

Hash code = 31965818


43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


Note that the instance that was returned still has the mutated state in the array from the last time the object was used. This may pose a security issue in some cases where it may be warranted to clear the contents of an object prior to returning it to the pool. To support this, we can implement additional interfaces on the pooled object type. We'll get to this in a moment.

#### Step 6 - Exploring pooling behavior a bit more

What happens if we allocate more objects than the specified `size` of the object pool? To figure this out, let's allocate a bunch more objects from the pool. But first, let's return `myObj1` to the pool.

In [1]:
myPool.Free(myObj1);

It goes without saying that `myObj1` should no longer be used after it has been returned to the pool because it can be used by some other piece of code at any time after having been returned. The use of `New` with a `using` statement makes it a bit harder to have this type of *use-after-free* bugs, but one should remain cautious and carefully review usage patterns of pooled objects.

Now, let's allocate more objects than fit in our pool. The pool's size is `4`, so let's allocate `5` objects.

In [1]:
var objs = new MyObject[5];

for (var i = 0; i < objs.Length; i++)
{
    objs[i] = myPool.Allocate();

    Console.WriteLine($"objs[{i}].GetHashCode() = {objs[i].GetHashCode()}");
}

objs[0].GetHashCode() = 31965818


objs[1].GetHashCode() = 12239935


Allocated a new MyObject instance. Hash code = 45407009


objs[2].GetHashCode() = 45407009


Allocated a new MyObject instance. Hash code = 46471022


objs[3].GetHashCode() = 46471022


Allocated a new MyObject instance. Hash code = 46242629


objs[4].GetHashCode() = 46242629


Note that the first two objects are reused from the `myObj1` and `myObj2` allocations we did earlier. The remainder three objects were freshly allocated. Now, we'll return all of them back to the pool.

In [1]:
for (var i = 0; i < objs.Length; i++)
{
    myPool.Free(objs[i]);
}

If we allocate another `5` objects now, we'll get four that get reused from the pool (because of its maximum size of `4`), while the fifth one will be new.

In [1]:
var objs = new MyObject[5];

for (var i = 0; i < objs.Length; i++)
{
    objs[i] = myPool.Allocate();

    Console.WriteLine($"objs[{i}].GetHashCode() = {objs[i].GetHashCode()}");
}

objs[0].GetHashCode() = 31965818


objs[1].GetHashCode() = 12239935


objs[2].GetHashCode() = 45407009


objs[3].GetHashCode() = 46471022


Allocated a new MyObject instance. Hash code = 45672827


objs[4].GetHashCode() = 45672827


#### Step 7 - Support clearing an instance upon returning to the pool

To support clearing an instance prior to returning it to the pool, we can implement the `IClearable` interface on `MyObject`.

In [1]:
class MyObject : IClearable
{
    // Demonstrates the expense of the object which may warrant pooling to reuse these array allocatons.
    private readonly int[] _values = new int[16];

    public int this[int i]
    {
        get => _values[i];
        set => _values[i] = value;
    }

    public void Clear() => Array.Clear(_values, 0, _values.Length);

    public override string ToString() => string.Join(", ", _values);
}

Let's create a new pool, just like we did before. We'll keep the side-effect in the factory delegate to spot reuse of objects further on.

In [1]:
using System.Memory;

var myPool = new ObjectPool<MyObject>(() =>
{
    var res = new MyObject();
    Console.WriteLine($"Allocated a new MyObject instance. Hash code = {res.GetHashCode()}");
    return res;
}, size: 4);

Finally, let's allocate an object, mutate it, return it to the pool, and then allocate another object. This will cause reuse of the object. However, this time around we should not see the result of mutating the array upon reusing the same instance, because `IClearable.Clear` has been called by the pool.

In [1]:
Console.WriteLine("First usage of the object");

using (var obj = myPool.New())
{
    Console.WriteLine($"obj#{obj.Object.GetHashCode()} = {obj.Object}");
    obj.Object[0] = 42;
    Console.WriteLine($"obj#{obj.Object.GetHashCode()} = {obj.Object}");
}

Console.WriteLine("Second usage of the object");

using (var obj = myPool.New())
{
    Console.WriteLine($"obj#{obj.Object.GetHashCode()} = {obj.Object}"); // Contents should be clear!
    obj.Object[0] = 42;
    Console.WriteLine($"obj#{obj.Object.GetHashCode()} = {obj.Object}");
}

First usage of the object


Allocated a new MyObject instance. Hash code = 39493100


obj#39493100 = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


obj#39493100 = 42, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


Second usage of the object


obj#39493100 = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


obj#39493100 = 42, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0


## Function memoization

Function memoization is a technique to cache the results of evaluating a pure function in order to speed up future invocations.

### A trivial example using the Fibonacci sequence

As an example, consider the well-known recursive Fibonacci generator.

In [1]:
Func<long, long> fib = null;

fib = n => n <= 1 ? 1 : checked(fib(n - 1) + fib(n - 2));

Evaluating the Fibonacci generator causes repeated evaluation of the same function with the same argument. For example:

```
fib(3) = fib(2) + fib(1)
fib(2) =          fib(1) + fib(0)
```

Let's run the Fibonacci generator for a few values and time the execution.

In [1]:
using System.Diagnostics;

void PrintFibonacci(int max, TimeSpan maxTimeToCompute)
{
    var sw = new Stopwatch();

    for (int i = 0; i < max; i++)
    {
        sw.Restart();

        long res = 0L;
        try
        {
            res = fib(i);
        }
        catch (OverflowException)
        {
            Console.WriteLine($"fib({i}) = Overflow");
            return;
        }

        sw.Stop();
        Console.WriteLine($"fib({i}) = {res} - Took {sw.Elapsed}");

        // Stop if it starts taking too long.
        if (sw.Elapsed > maxTimeToCompute)
        {
            Console.WriteLine($"Aborted at iteration {i}. This is starting to take too long.");
            break;
        }
    }
}

PrintFibonacci(100, TimeSpan.FromSeconds(5));

fib(0) = 1 - Took 00:00:00.0000998


fib(1) = 1 - Took 00:00:00.0000002


fib(2) = 2 - Took 00:00:00.0000001


fib(3) = 3 - Took 00:00:00.0000004


fib(4) = 5 - Took 00:00:00.0000001


...

fib(40) = 165580141 - Took 00:00:01.3919132


fib(41) = 267914296 - Took 00:00:02.1608034


fib(42) = 433494437 - Took 00:00:03.1233396


fib(43) = 701408733 - Took 00:00:05.2934294


Aborted at iteration 43. This is starting to take too long.


Most likely, you didn't get much further than some `40`-ish iterations. Let's use this example to illustrate memoization for function evaluation.

The first step to make memoization work is to create a so-called *memoization cache factory*. Each memoized function will have an associated memoization cache. Factories for such caches determine the policy of the cache. In the sample below we'll use an `Unbounded` cache which does not limit the number of entries in the cache. Other options are caches with least-recently-used (LRU) policies or other eviction policies.

In [1]:
using System.Memory;

IMemoizationCacheFactory factory = MemoizationCacheFactory.Unbounded;

Now that we have a cache factory, we can create a *memoizer* that will be used to memoize functions.

In [1]:
IMemoizer mem = Memoizer.Create(factory);

Finally, we use the memoizer to `Memoize` the function. After doing so, we end up with a pair of a cache and a memoized function of the same delegate type.

In [1]:
IMemoizedDelegate<Func<long, long>> memoizedDelegateFib = mem.Memoize(fib);

// The cache and delegate pair.
IMemoizationCache cache = memoizedDelegateFib.Cache;
Func<long, long> fibMemoized = memoizedDelegateFib.Delegate;

// Let's replace the original delegate by the memoized one, which was also used in the body of the recursive definition of fib.
fib = fibMemoized;

// Now we should get much further along.
PrintFibonacci(100, TimeSpan.FromSeconds(5));

fib(0) = 1 - Took 00:00:00.0036124


fib(1) = 1 - Took 00:00:00.0000302


fib(2) = 2 - Took 00:00:00.0001916


fib(3) = 3 - Took 00:00:00.0006244


fib(4) = 5 - Took 00:00:00.0000033


...

fib(88) = 1779979416004714189 - Took 00:00:00.0000003


fib(89) = 2880067194370816120 - Took 00:00:00.0000088


fib(90) = 4660046610375530309 - Took 00:00:00.0000004


fib(91) = 7540113804746346429 - Took 00:00:00.0000004


fib(92) = Overflow


To see what's going on, let's explore the cache.

In [1]:
cache.DebugView

Number of entries = 92


While the output of `DebugView` is a bit spartan, note that we have `92` entries which contain the values of evaluating `fib(0)` through `fib(91)`. We can also go ahead and clear the cache manually, using the `Clear` method.

**Note:** The use of `Clear` is atypical but is sometimes useful after performing a lot of operations in a certain "phase" of execution in a program and where it makes sense to reclaim resources. Caches in Nuqleon are not actively maintained; there are no background threads or timers to prune caches when they're not in use. However, simply dropping the reference to the memoized delegate will also cause the cache to get garbage collected. That's often a more convenient approach to manage caches.

In [1]:
cache.Clear();

cache.DebugView

Number of entries = 0


### Exploring memoization in more detail

To explore what's going on, let's craft a more sophisticated example using an instrumented function.

In [1]:
using System.Threading;

static double GetRadius(double x, double y)
{
    Console.WriteLine($"GetRadius({x}, {y}) was called");
    Thread.Sleep(1000);
    return Math.Sqrt(x * x + y * y);
}

Invoking this function directly takes a little over a second to complete, mimicking the expense of a real function.

In [1]:
var sw = Stopwatch.StartNew();
Console.WriteLine($"GetRadius(3, 4) = {GetRadius(3, 4)} in {sw.Elapsed}");

GetRadius(3, 4) was called


GetRadius(3, 4) = 5 in 00:00:01.0052418


By using memoization, we can cache and reuse the result. This time around, we'll create an LRU cache to explore cache policies.

In [1]:
using System.Memory;

IMemoizationCacheFactory factory = MemoizationCacheFactory.CreateLru(maxCapacity: 4);

IMemoizer memoizer = Memoizer.Create(factory);

Unlike our Fibonacci example, we start off with a method here, rather than a delegate. Furthermore, we have more than one parameter in this case. To pick the right overload of `Memoize`, we will be explicit about the parameter and result types. As a result, we'll end up with a `Func<double, double, double>` delegate that represents the memoized `GetRadius` method. While we're at it, we'll also explore other parameters of `Memoize`, all of which are optional and have suitable defaults.

In [1]:
IMemoizedDelegate<Func<double, double, double>> getRadiusMemoized = memoizer.Memoize<double, double, double>(GetRadius, MemoizationOptions.CacheException, EqualityComparer<double>.Default, EqualityComparer<double>.Default);

The first additional parameter is a `MemoizationOptions` enum which enables turning on caching of exceptions in case the function throws. This is off by default. In our example, this is obviously quite useless. The additional two parameters are `IEqualityComparer<T>` instances for the two inputs of the `GetRadius` function. These are used to look up existing `x, y` pairs in the cache when trying to find a match. An example where this can be useful is for functions that take in an array and one wants to check the array for element-wise equality.

**Note:** A concrete example of memoization is in `Nuqleon.Reflection.Virtualization` where expensive reflection calls get memoized. This touches on various design points mentioned here. For example, some APIs may throw an exception, and we may want to cache these. Also, APIs like `MakeGenericType` take in a `Type[]` and memoization requires a way to compare two such arrays for element-wise equality.

With the resulting memoized delegate, we can now see the behavior of repeated invocation of `GetRadius` with memoization applied.

In [1]:
var sw = Stopwatch.StartNew();
Console.WriteLine($"GetRadius(3, 4) = {getRadiusMemoized.Delegate(3, 4)} in {sw.Elapsed}");

sw.Restart();
Console.WriteLine($"GetRadius(3, 4) = {getRadiusMemoized.Delegate(3, 4)} in {sw.Elapsed}");

GetRadius(3, 4) was called


GetRadius(3, 4) = 5 in 00:00:01.0102280


GetRadius(3, 4) = 5 in 00:00:00.0003059


Note that the second invocation did not trigger the invocation of `GetRadius` and served up the result from the cache. Let's now print the cache's `DebugView`, clear the cache, and try to invoke the memoized delegate again.

In [1]:
Console.WriteLine(getRadiusMemoized.Cache.DebugView);

getRadiusMemoized.Cache.Clear();

sw.Restart();
Console.WriteLine($"GetRadius(3, 4) = {getRadiusMemoized.Delegate(3, 4)} in {sw.Elapsed}");

sw.Restart();
Console.WriteLine($"GetRadius(3, 4) = {getRadiusMemoized.Delegate(3, 4)} in {sw.Elapsed}");

Entries
-------

  (3, 4) -> 5
    Invoke duration     = 00:00:01.0013880
    Hit count           = 2
    Average access time = 00:00:00.5047844
    Speed up factor     = 1.9837934769774976

Summary
-------

  Invocation count = 1
  Access count     = 2
  Eviction count   = 0




GetRadius(3, 4) was called


GetRadius(3, 4) = 5 in 00:00:01.0153758


GetRadius(3, 4) = 5 in 00:00:00.0000438


Note that the `DebugView` output is much more verbose. This is because we're now using an LRU cache which has a much more elaborate `DebugView` to analyze what's going on. For our initial exploration, keep an eye on `Eviction count`, which will reflect the LRU behavior where the least recently used entry gets evicted from the cache. To illustrate this, let's invoke the memoized delegate with various inputs.

In [1]:
for (int i = 0; i < 2; i++)
{
    foreach (var (x, y) in new[] {
        (1, 2),
        (2, 3),
        (3, 4),
    })
    {
        sw.Restart();
        Console.WriteLine($"GetRadius({x}, {y}) = {getRadiusMemoized.Delegate(x, y)} in {sw.Elapsed}");
    }
}

Console.WriteLine(getRadiusMemoized.Cache.DebugView);

GetRadius(1, 2) was called


GetRadius(1, 2) = 2.23606797749979 in 00:00:01.0120257


GetRadius(2, 3) was called


GetRadius(2, 3) = 3.605551275463989 in 00:00:01.0046993


GetRadius(3, 4) = 5 in 00:00:00.0000081


GetRadius(1, 2) = 2.23606797749979 in 00:00:00.0000033


GetRadius(2, 3) = 3.605551275463989 in 00:00:00.0000029


GetRadius(3, 4) = 5 in 00:00:00.0000023


Entries
-------

  (3, 4) -> 5
    Invoke duration     = 00:00:01.0152318
    Hit count           = 4
    Average access time = 00:00:00.2538540
    Speed up factor     = 3.999274386064431

  (2, 3) -> 3.605551275463989
    Invoke duration     = 00:00:01.0046303
    Hit count           = 2
    Average access time = 00:00:00.5023476
    Speed up factor     = 1.9998708065889037

  (1, 2) -> 2.23606797749979
    Invoke duration     = 00:00:01.0119472
    Hit count           = 2
    Average access time = 00:00:00.5060107
    Speed up factor     = 1.999853362784621

Summary
-------

  Invocation count = 3
  Access count     = 8
  Eviction count   = 0




Because we've only invoked the function with three distinct input pairs, we never ended up causing any eviction. The order of the entries in the `DebugView` shows the latest invocation at the top. Let's make another invocation for a unique input.

In [1]:
sw.Restart();
Console.WriteLine($"GetRadius(4, 5) = {getRadiusMemoized.Delegate(4, 5)} in {sw.Elapsed}");

Console.WriteLine(getRadiusMemoized.Cache.DebugView);

GetRadius(4, 5) was called


GetRadius(4, 5) = 6.4031242374328485 in 00:00:01.0039645


Entries
-------

  (4, 5) -> 6.4031242374328485
    Invoke duration     = 00:00:01.0026460
    Hit count           = 1
    Average access time = 00:00:01.0039598
    Speed up factor     = 0.9986913818660866 [DEGRADATION]

  (3, 4) -> 5
    Invoke duration     = 00:00:01.0152318
    Hit count           = 4
    Average access time = 00:00:00.2538540
    Speed up factor     = 3.999274386064431

  (2, 3) -> 3.605551275463989
    Invoke duration     = 00:00:01.0046303
    Hit count           = 2
    Average access time = 00:00:00.5023476
    Speed up factor     = 1.9998708065889037

  (1, 2) -> 2.23606797749979
    Invoke duration     = 00:00:01.0119472
    Hit count           = 2
    Average access time = 00:00:00.5060107
    Speed up factor     = 1.999853362784621

Summary
-------

  Invocation count = 4
  Access count     = 9
  Eviction count   = 0




Now, the cache is full. To see the order of the entries change, we can make some more invocations with these input pairs. All accesses will be sped up because they get served from the cache.

In [1]:
foreach (var (x, y) in new[] {
    (4, 5),
    (1, 2),
    (2, 3),
    (3, 4),
})
{
    sw.Restart();
    Console.WriteLine($"GetRadius({x}, {y}) = {getRadiusMemoized.Delegate(x, y)} in {sw.Elapsed}");
}

Console.WriteLine(getRadiusMemoized.Cache.DebugView);

GetRadius(4, 5) = 6.4031242374328485 in 00:00:00.0000144


GetRadius(1, 2) = 2.23606797749979 in 00:00:00.0000095


GetRadius(2, 3) = 3.605551275463989 in 00:00:00.0000017


GetRadius(3, 4) = 5 in 00:00:00.0000030


Entries
-------

  (3, 4) -> 5
    Invoke duration     = 00:00:01.0152318
    Hit count           = 5
    Average access time = 00:00:00.2030838
    Speed up factor     = 4.999078213033241

  (2, 3) -> 3.605551275463989
    Invoke duration     = 00:00:01.0046303
    Hit count           = 3
    Average access time = 00:00:00.3348989
    Speed up factor     = 2.9998017312090304

  (1, 2) -> 2.23606797749979
    Invoke duration     = 00:00:01.0119472
    Hit count           = 3
    Average access time = 00:00:00.3373434
    Speed up factor     = 2.999753959911473

  (4, 5) -> 6.4031242374328485
    Invoke duration     = 00:00:01.0026460
    Hit count           = 2
    Average access time = 00:00:00.5019845
    Speed up factor     = 1.9973644604564484

Summary
-------

  Invocation count = 4
  Access count     = 13
  Eviction count   = 0




Now, the input pair `(4, 5)` is the least recently used one. Let's try to invoke the memoized function with a new unique input pair, and see this entry getting evicted.

In [1]:
sw.Restart();
Console.WriteLine($"GetRadius(5, 6) = {getRadiusMemoized.Delegate(5, 6)} in {sw.Elapsed}");

Console.WriteLine(getRadiusMemoized.Cache.DebugView);

GetRadius(5, 6) was called


GetRadius(5, 6) = 7.810249675906654 in 00:00:01.0163047


Entries
-------

  (5, 6) -> 7.810249675906654
    Invoke duration     = 00:00:01.0150146
    Hit count           = 1
    Average access time = 00:00:01.0162988
    Speed up factor     = 0.998736395241242 [DEGRADATION]

  (3, 4) -> 5
    Invoke duration     = 00:00:01.0152318
    Hit count           = 5
    Average access time = 00:00:00.2030838
    Speed up factor     = 4.999078213033241

  (2, 3) -> 3.605551275463989
    Invoke duration     = 00:00:01.0046303
    Hit count           = 3
    Average access time = 00:00:00.3348989
    Speed up factor     = 2.9998017312090304

  (1, 2) -> 2.23606797749979
    Invoke duration     = 00:00:01.0119472
    Hit count           = 3
    Average access time = 00:00:00.3373434
    Speed up factor     = 2.999753959911473

Summary
-------

  Invocation count = 5
  Access count     = 14
  Eviction count   = 1

  Last eviction
    Invoke duration     = 00:00:01.0026460
    Hit count           = 2
    Average access time = 00:00:00.5019845
    Speed u

The eviction count is `1` now. If we try to invoke the delegate with inputs `(4, 5)` again, we'll see `GetRadius` getting invoked again. This time, `(1, 2)` is the least recently used entry which will get evicted.

In [1]:
sw.Restart();
Console.WriteLine($"GetRadius(4, 5) = {getRadiusMemoized.Delegate(4, 5)} in {sw.Elapsed}");

Console.WriteLine(getRadiusMemoized.Cache.DebugView);

GetRadius(4, 5) was called


GetRadius(4, 5) = 6.4031242374328485 in 00:00:01.0060432


Entries
-------

  (4, 5) -> 6.4031242374328485
    Invoke duration     = 00:00:01.0059979
    Hit count           = 1
    Average access time = 00:00:01.0060376
    Speed up factor     = 0.9999605382542363 [DEGRADATION]

  (5, 6) -> 7.810249675906654
    Invoke duration     = 00:00:01.0150146
    Hit count           = 1
    Average access time = 00:00:01.0162988
    Speed up factor     = 0.998736395241242 [DEGRADATION]

  (3, 4) -> 5
    Invoke duration     = 00:00:01.0152318
    Hit count           = 5
    Average access time = 00:00:00.2030838
    Speed up factor     = 4.999078213033241

  (2, 3) -> 3.605551275463989
    Invoke duration     = 00:00:01.0046303
    Hit count           = 3
    Average access time = 00:00:00.3348989
    Speed up factor     = 2.9998017312090304

Summary
-------

  Invocation count = 6
  Access count     = 15
  Eviction count   = 2

  Last eviction
    Invoke duration     = 00:00:01.0119472
    Hit count           = 3
    Average access time = 00:00:00.33

### More advanced cache policies

In the samples above we've seen the use of an unbounded and an LRU-based cache for memoization. The `Nuqleon.Memory` library also supports more advanced cache management schemes.

A first example is the use of `CreateEvictedBy[Highest|Lowest]` for any metric on `IMemoizationCacheEntryMetrics`. In fact, the LRU policy is merely performing an eviction based on the `LastAccessTime` metric that's kept for entries in the cache. In the sample below, we'll use the `SpeedUpFactory` metric which represents a ratio between the time it took to invoke the function for the given arguments, prior to caching the result, and the time taken by subsequent invocations, served from the cache.

In [1]:
static int GetValueDelayed(int x, int ms)
{
    Thread.Sleep(ms);
    return x;
}

The `GetValueDelayed` function illustrates the difference in time needed to invoke a function based on its arguments. We can now memoize the function using `CreateEvictedByLowest` using the `SpeedupFactory` metric. We'll also limit the cache to 4 entries using the `maxCapacity` parameter.

**Note:** The `ageThreshold` parameter is slightly more complex. Every time the memoization cache gets accessed, the cache entry that was used to satisfy the request (i.e. either an existing entry or a freshly created one) is moved to the top of an internal data structure. This keeps them ordered by the last access time, which is directly usable for LRU policies. When an eviction has to be made based on another metric, the tail of this list of entries is used to find a candidate, excluding the most recent items. This is done to give recent items a chance to get more statistically relevant data, especially for new entries that shouldn't get evicted immediately. The `ageThreshold` specifies this cut-off point. By setting it to `1.0` rather than the default of `0.9`, we will consider all cache entries valid to be valid as eviction candidates.

In [1]:
var factory = MemoizationCacheFactory.CreateEvictedByLowest(metric => metric.SpeedupFactor, maxCapacity: 4, ageThreshold: 1.0);

var memoizer = Memoizer.Create(factory);

var getValueDelayedMemoized = memoizer.Memoize<int, int, int>(GetValueDelayed);

Also note that the delegate passed to `CreateEvictedByLowest` can contain any computation based on the given metrics, so users are free to compute other derived metrics in case the built-in ones do not meet certain criteria. Using this delegate it's also possible to create a random eviction policy, simply by returning a random number.

Now we'll go ahead and invoke the memoized function for a few times different inputs which will cause the computation of metrics for each entry, as shown by dumping the cache's `DebugView`.

In [1]:
for (int i = 0; i < 100; i++)
{
    for (int j = 1; j <= 4; j++)
    {
        getValueDelayedMemoized.Delegate(42, j * 10);
    }
}

Console.WriteLine(getValueDelayedMemoized.Cache.DebugView);

Entries
-------

  (42, 10) -> 42
    Invoke duration     = 00:00:00.0223207
    Hit count           = 100
    Average access time = 00:00:00.0002765
    Speed up factor     = 80.72585895117541

  (42, 20) -> 42
    Invoke duration     = 00:00:00.0283414
    Hit count           = 100
    Average access time = 00:00:00.0002853
    Speed up factor     = 99.33894146512444

  (42, 30) -> 42
    Invoke duration     = 00:00:00.0315064
    Hit count           = 100
    Average access time = 00:00:00.0003152
    Speed up factor     = 99.95685279187818

  (42, 40) -> 42
    Invoke duration     = 00:00:00.0480687
    Hit count           = 100
    Average access time = 00:00:00.0004882
    Speed up factor     = 98.46108152396559

Summary
-------

  Invocation count = 4
  Access count     = 400
  Trim count       = 0
  Trim elapsed     = 00:00:00
  Eviction count   = 0




The reported speed up factor for the different entries will differ slightly after the decimal point, where the 40ms invocation of the function has the highest speed up and the 10ms invocation of the function has the lowest speed up. Upon doing a new invocation that requires the eviction of an entry, the entry with the lowest speed up will be evicted. In the cell below, we invoke the function with a different argument value to cause eviction.

In [1]:
getValueDelayedMemoized.Delegate(43, 10);

Console.WriteLine(getValueDelayedMemoized.Cache.DebugView);

Entries
-------

  (43, 10) -> 43
    Invoke duration     = 00:00:00.0243194
    Hit count           = 1
    Average access time = 00:00:00.0279688
    Speed up factor     = 0.8695188924801922 [DEGRADATION]

  (42, 20) -> 42
    Invoke duration     = 00:00:00.0283414
    Hit count           = 100
    Average access time = 00:00:00.0002853
    Speed up factor     = 99.33894146512444

  (42, 30) -> 42
    Invoke duration     = 00:00:00.0315064
    Hit count           = 100
    Average access time = 00:00:00.0003152
    Speed up factor     = 99.95685279187818

  (42, 40) -> 42
    Invoke duration     = 00:00:00.0480687
    Hit count           = 100
    Average access time = 00:00:00.0004882
    Speed up factor     = 98.46108152396559

Summary
-------

  Invocation count = 5
  Access count     = 401
  Trim count       = 1
  Trim elapsed     = 00:00:00.0036386
  Eviction count   = 1

  Last eviction
    Invoke duration     = 00:00:00.0223207
    Hit count           = 100
    Average access 

Memoization caches also support direct trimming using an interface called `ITrimmable`. Different types of trimming are possible, for example based on metrics. This is illustrated in the cell below where we drop cache entries with a `HitCount` less than `10`.

In [1]:
int trimCount = getValueDelayedMemoized.Cache.ToTrimmableByMetrics().Trim(metric => metric.HitCount < 10);
Console.WriteLine($"Trimmed {trimCount} entries.");

Console.WriteLine(getValueDelayedMemoized.Cache.DebugView);

Trimmed 1 entries.


Entries
-------

  (42, 20) -> 42
    Invoke duration     = 00:00:00.0283414
    Hit count           = 100
    Average access time = 00:00:00.0002853
    Speed up factor     = 99.33894146512444

  (42, 30) -> 42
    Invoke duration     = 00:00:00.0315064
    Hit count           = 100
    Average access time = 00:00:00.0003152
    Speed up factor     = 99.95685279187818

  (42, 40) -> 42
    Invoke duration     = 00:00:00.0480687
    Hit count           = 100
    Average access time = 00:00:00.0004882
    Speed up factor     = 98.46108152396559

Summary
-------

  Invocation count = 5
  Access count     = 401
  Trim count       = 1
  Trim elapsed     = 00:00:00.0036386
  Eviction count   = 1

  Last eviction
    Invoke duration     = 00:00:00.0223207
    Hit count           = 100
    Average access time = 00:00:00.0002765
    Speed up factor     = 80.72585895117541




### Thread safety of memoization caches

By default, memoization caches returned from memoization cache factories are **not** thread-safe. This is a deliberate design choice in order to avoid overheads for single-threaded scenarios. In order to create thread-safe memoizers, one can use a few different approaches.

* Use `ConcurrentMemoizationCache` instead of `MemoizationCache`.
* Use the `Synchronized` extension method on memoization cache factories.
* Use the `WithThreadLocal` extension method on memoization cache factories.

All of these return an `IMemoizationCacheFactory` that produces caches with thread-safe behavior. Alternatively, one can memoize the same function multiple times, on different threads, and ensure that only that thread calls the memoized function.

In the sample below, we use `WithThreadLocal` to cause memoization caches to be allocated on each distinct thread.

In [1]:
using System.Memory;

var factory = MemoizationCacheFactory.CreateLru(maxCapacity: 8).WithThreadLocal();

var memoizer = Memoizer.Create(factory);

var f = memoizer.Memoize((int x) =>
{
    Console.WriteLine($"~{Environment.CurrentManagedThreadId} - f({x})");
    return x + 1;
});

Next, let's use the cache from two different threads. Each of them will have its own cache.

In [1]:
using System.Threading;

var t1 = new Thread(() =>
{
    f.Delegate(1);
    f.Delegate(2);
    f.Delegate(1); // used from thread-local cache

    Console.WriteLine(f.Cache.DebugView);
});
t1.Start();
t1.Join();

var t2 = new Thread(() =>
{
    f.Delegate(1); // unique cache on this thread
    f.Delegate(2);
    f.Delegate(1); // used from thread-local cache

    Console.WriteLine(f.Cache.DebugView);
});
t2.Start();
t2.Join();

~28 - f(1)


~28 - f(2)


Entries
-------

  1 -> 2
    Invoke duration     = 00:00:00.0003984
    Hit count           = 2
    Average access time = 00:00:00.0022092
    Speed up factor     = 0.18033677349266702 [DEGRADATION]

  2 -> 3
    Invoke duration     = 00:00:00.0001152
    Hit count           = 1
    Average access time = 00:00:00.0001554
    Speed up factor     = 0.7413127413127413 [DEGRADATION]

Summary
-------

  Invocation count = 2
  Access count     = 3
  Eviction count   = 0




~7 - f(1)


~7 - f(2)


Entries
-------

  1 -> 2
    Invoke duration     = 00:00:00.0000585
    Hit count           = 2
    Average access time = 00:00:00.0000394
    Speed up factor     = 1.484771573604061

  2 -> 3
    Invoke duration     = 00:00:00.0000154
    Hit count           = 1
    Average access time = 00:00:00.0000176
    Speed up factor     = 0.875 [DEGRADATION]

Summary
-------

  Invocation count = 2
  Access count     = 3
  Eviction count   = 0




### Intern caches

Intern caches are often used to deduplicate instances of immutable objects based on value equality. The best known sample is `string.Intern(string)` which deduplicates strings. For example, the result of calling `"BAR".ToLower()` can get deduplicated by `string.Intern` if an existing string with contents `"bar"` exists. The old copy with identical contents can then be garbage collected.

Memoization caches can be used to construct intern caches, simply by memoizing an identity function `(T x) => x` using an `IEqualityComparer<T>` that checks for value equality. As an example, let's build an intern cache for `ReadOnlyCollection<T>` objects. First, we'll create an `IEqualityComparer<ReadOnlyCollection<T>>` implementation for such immutable collections, using pairwise element equality.

In [1]:
using System.Collections.Generic;
using System.Linq;

class SequenceEqualityComparer<T> : IEqualityComparer<IEnumerable<T>>
{
    public bool Equals(IEnumerable<T> xs, IEnumerable<T> ys)
    {
        if (xs is null)
        {
            return ys is null;
        }

        if (ys is null)
        {
            return false;
        }

        return xs.SequenceEqual(ys);
    }

    public int GetHashCode(IEnumerable<T> xs)
    {
        HashCode h = new();

        if (xs is not null)
        {
            foreach (var x in xs)
            {
                h.Add(x);
            }
        }

        return h.ToHashCode();
    }
}

Next, we'll create an intern cache.

In [1]:
using System.Collections.ObjectModel;
using System.Memory;

IInternCache<ReadOnlyCollection<int>> cache = MemoizationCacheFactory.Unbounded.CreateInternCache<ReadOnlyCollection<int>>(new SequenceEqualityComparer<int>());

Finally, we can try out our cache by instantiating multiple copies of a `ReadOnlyCollection<int>` with the same contents and running them through `Intern`.

In [1]:
var xs = new ReadOnlyCollection<int>(Enumerable.Range(0, 10).ToArray());

Console.WriteLine($"xs.GetHashCode() = {xs.GetHashCode()}");

xs = cache.Intern(xs);

Console.WriteLine($"xs.GetHashCode() = {xs.GetHashCode()} after interning");

var ys = new ReadOnlyCollection<int>(Enumerable.Range(0, 10).ToArray());

Console.WriteLine($"ys.GetHashCode() = {ys.GetHashCode()}");

ys = cache.Intern(ys);

Console.WriteLine($"ys.GetHashCode() = {ys.GetHashCode()} after interning");

xs.GetHashCode() = 11261288


xs.GetHashCode() = 11261288 after interning


ys.GetHashCode() = 37913673


ys.GetHashCode() = 11261288 after interning
